1. Creating a S3 Bucket

In [1]:
import boto3
s3 = boto3.resource('s3')

In [8]:
# Set the S3 bucket name and the region
bucket_name = 'globeossstockprices'  # Replace with your desired bucket name
region = 'ap-southeast-1'

# Create an S3 client
s3_client = boto3.client('s3', region_name=region)

# Create the S3 bucket
try:
    response = s3_client.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={
            'LocationConstraint': region
        }
    )
    print(f'Bucket {bucket_name} created successfully in {region}.')
except Exception as e:
    print(f'S3 Error: {e}')


Bucket globeossstockprices created successfully in ap-southeast-1.


In [9]:
!pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 46.8 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 2.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 90.3 MB/s eta 0:00:00:00:01
  Created wheel for peewee: filename=peewee-3.17.6-cp310-cp310-linux_x86_64.whl size=279105 sha256=5afb412561ab90bf4b12ebe6469097c74bd9e9e1124b22f3da8ba1186c3f1bfb
  Stored in directory: /home/ec2-user/.cache/pip/wheels/4b/b9/b0/83d6e258e8f963f5ff111a2cd8c483ca59372a86e6a2535212
Successfully built peewee


In [10]:
import pandas as pd 
from datetime import datetime
import yfinance as yf

#initialize parameters
start_date = datetime(2019,1,1)
end_date = datetime(2021,1,1)

#get the data
df_data = yf.download('AWV.SI', start = start_date, end = end_date)

df_data.reset_index(inplace=True)
df_data

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-01,0.37,0.37,0.370,0.370,0.315062,0
1,2019-01-02,0.37,0.37,0.370,0.370,0.315062,0
2,2019-01-03,0.37,0.37,0.370,0.370,0.315062,0
3,2019-01-04,0.37,0.37,0.370,0.370,0.315062,0
4,2019-01-07,0.37,0.37,0.370,0.370,0.315062,0
...,...,...,...,...,...,...,...
500,2020-12-24,0.34,0.34,0.340,0.340,0.309417,0
501,2020-12-28,0.34,0.34,0.340,0.340,0.309417,0
502,2020-12-29,0.34,0.34,0.340,0.340,0.309417,0
503,2020-12-30,0.35,0.35,0.325,0.335,0.304867,65000


Extract, Load & Transform

In [12]:
df_data.drop(axis=1, columns=['Adj Close'], inplace= True)
df_data.drop(axis=1, columns=['Date'], inplace= True)

In [13]:
df_data_features = df_data.iloc[:-1 , :]
df_data_features

,Open,High,Low,Close,Volume
0,0.37,0.37,0.370,0.370,0
1,0.37,0.37,0.370,0.370,0
2,0.37,0.37,0.370,0.370,0
3,0.37,0.37,0.370,0.370,0
4,0.37,0.37,0.370,0.370,0
...,...,...,...,...,...
499,0.34,0.34,0.340,0.340,0
500,0.34,0.34,0.340,0.340,0
501,0.34,0.34,0.340,0.340,0
502,0.34,0.34,0.340,0.340,0


In [14]:
df_data_targets = df_data.iloc[1:, 0].rename("Targets")
df_data_targets

1      0.370
2      0.370
3      0.370
4      0.370
5      0.385
       ...  
500    0.340
501    0.340
502    0.340
503    0.350
504    0.350
Name: Targets, Length: 504, dtype: float64

In [15]:
df_data_features['Target'] = list(df_data_targets)

first_column = df_data_features.pop('Target')
df_data_features.insert(0, 'Target', first_column)

df_data_final = df_data_features
df_data_final

/tmp/ipykernel_14425/2375105290.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_features['Target'] = list(df_data_targets)


,Target,Open,High,Low,Close,Volume
0,0.370,0.37,0.37,0.370,0.370,0
1,0.370,0.37,0.37,0.370,0.370,0
2,0.370,0.37,0.37,0.370,0.370,0
3,0.370,0.37,0.37,0.370,0.370,0
4,0.385,0.37,0.37,0.370,0.370,0
...,...,...,...,...,...,...
499,0.340,0.34,0.34,0.340,0.340,0
500,0.340,0.34,0.34,0.340,0.340,0
501,0.340,0.34,0.34,0.340,0.340,0
502,0.350,0.34,0.34,0.340,0.340,0


In [17]:
import numpy as np

df_randomized = df_data_final.sample(frac=1, random_state=123)

Train Test Split

In [18]:
train_data, test_data = np.split(df_randomized, [int(0.8*len(df_randomized))])
print(train_data.shape, test_data.shape)

(403, 6) (101, 6)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Set a path and upload dataset to S3 bucket

In [19]:
import os

prefix = 'xgboost-as-a-built-in-algo'

train_csv_path = 's3://{}/{}/{}/{}'.format(bucket_name, prefix, 'train', 'train.csv')
test_csv_path = 's3://{}/{}/{}/{}'.format(bucket_name, prefix, 'test', 'test.csv')

print(train_csv_path)
print(test_csv_path)

s3://globeossstockprices/xgboost-as-a-built-in-algo/train/train.csv
s3://globeossstockprices/xgboost-as-a-built-in-algo/test/test.csv


In [22]:
train_data.to_csv(train_csv_path, index = False, header =False)
test_data.to_csv(test_csv_path, index = False, header = False)

In [21]:
!pip install --upgrade s3fs


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.2/77.2 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 104.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.0
    Uninstalling fsspec-2024.6.0:
      Successfully uninstalled fsspec-2024.6.0
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.142
    Uninstalling botocore-1.34.142:
      Successfully uninstalled botocore-1.34.142
  Attempting uninstall: s3fs
    Found existing installation: s3fs 0.4.2
    Uninstalling s3fs-0.4.2:
      Successfully uninstalled s3fs-0.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.33.24 requires botocore==1.34.142, but you have botocore 1.34.131 which is incompatible.
bot

3. Build XGBoost Model

Find a XGBoost Image URI and builds and XGBoost container

In [23]:
import sagemaker
from sagemaker import image_uris
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [24]:
xgboost_container = image_uris.retrieve("xgboost", boto3.Session().region_name, "1.2-2")
display(xgboost_container)

'121021644041.dkr.ecr.ap-southeast-1.amazonaws.com/sagemaker-xgboost:1.2-2'

In [25]:
hyperparameters = {
    "max_depth":"5",
    "eta":"0.2",
    "gamma":"4",
    "min_child_weight":"6",
    "subsample":"0.7",
    "objective":"reg:squarederror",
    "early_stopping_rounds":10,
    "num_round":1000
}

In [27]:
output_path = 's3://{}/{}/{}/'.format(bucket_name, prefix, 'output')
print(output_path)

s3://globeossstockprices/xgboost-as-a-built-in-algo/output/


In [28]:
estimator = sagemaker.estimator.Estimator(image_uri = xgboost_container,
                                          hyperparameters = hyperparameters,
                                          role = sagemaker.get_execution_role(),
                                          instance_count = 1,
                                          instance_type = 'ml.m4.xlarge',
                                          volume_size = 5, # 5 GB
                                          output_path = output_path,
                                          use_spot_instances = True,
                                          max_run = 300,
                                          max_wait = 600
                                         )


In [29]:
content_type = "csv"
train_input = TrainingInput("s3://{}/{}/{}/".format(bucket_name, prefix, 'train'), content_type=content_type)
test_input = TrainingInput("s3://{}/{}/{}/".format(bucket_name, prefix, 'test'), content_type=content_type)


In [30]:
estimator.fit({'train': train_input, 'validation': test_input})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-08-13-08-59-54-482


2024-08-13 08:59:54 Starting - Starting the training job...
2024-08-13 09:00:19 Starting - Preparing the instances for training......
2024-08-13 09:01:07 Downloading - Downloading input data...
2024-08-13 09:01:33 Downloading - Downloading the training image......
2024-08-13 09:02:49 Training - Training image download completed. Training in progress..[2024-08-13 09:02:57.136 ip-10-0-202-96.ap-southeast-1.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-08-13:09:02:57:INFO] Imported framework sagemaker_xgboost_container.training
[2024-08-13:09:02:57:INFO] Failed to parse hyperparameter objective value reg:squarederror to Json.
Returning the value itself
[2024-08-13:09:02:57:INFO] No GPUs detected (normal if no gpus installed)
[2024-08-13:09:02:57:INFO] Running XGBoost Sagemaker in algorithm mode
[2024-08-13:09:02:57:INFO] Determined delimiter of CSV input is ','
[2024-08-13:09:02:57:INFO] Determined delimiter of CSV input is ','
[2024-08-13:09:02:57:INFO] D

Deploying trained XGB model as Endpoint

In [32]:
from sagemaker.serializers import CSVSerializer

xgb_predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge', serializer=CSVSerializer())

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-08-13-09-11-17-472
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-08-13-09-11-17-472
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-08-13-09-11-17-472


------!

In [33]:
xgb_predictor.endpoint_name

'sagemaker-xgboost-2024-08-13-09-11-17-472'

In [34]:
# initialize parameters
start_date = datetime(2021, 1, 4)
end_date = datetime(2021, 1, 5)

# get the data
df_data = yf.download('AWV.SI', start = start_date, end = end_date)
df_data.reset_index(inplace=True)
df_data

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-01-04,0.325,0.325,0.325,0.325,0.295766,20000


In [35]:
df_data.drop(axis=1, columns=['Adj Close'], inplace=True)
df_data.drop(axis=1, columns=['Date'], inplace=True)

data_features_array = df_data.values
data_features_array

array([[3.24999988e-01, 3.24999988e-01, 3.24999988e-01, 3.24999988e-01,
        2.00000000e+04]])

In [36]:
Y_pred_Fcn = xgb_predictor.predict(data_features_array).decode('utf-8')


In [39]:
from sagemaker.serializers import CSVSerializer

Serialized_Input_Fcn = CSVSerializer().serialize([[3.24999988e-01, 3.24999988e-01, 3.24999988e-01, 3.24999988e-01, 2.00000000e+04]])

print(Serialized_Input_Fcn, type(Serialized_Input_Fcn))
Y_pred_Fcn = xgb_predictor.predict(Serialized_Input_Fcn).decode('utf-8')
Y_pred_Fcn

0.324999988,0.324999988,0.324999988,0.324999988,20000.0 <class 'str'>


'0.36412209272384644\n'

In [40]:
Input = [[0.324999988,0.324999988,0.324999988,0.324999988,20000.0],
        [0.324999988,0.324999988,0.324999988,0.324999988,20000.0],
        [0.324999988,0.324999988,0.324999988,0.324999988,20000.0]]

Serialized_Input = ','.join(map(str, Input[0]))

print(Serialized_Input_Fcn, type(Serialized_Input_Fcn))


0.324999988,0.324999988,0.324999988,0.324999988,20000.0 <class 'str'>


Lambda Function Handler

In [54]:
import boto3

ENDPOINT_NAME = 'sagemaker-xgboost-2024-08-13-09-11-17-472'
runtime = boto3.client('runtime.sagemaker')

def lambda_handler(event, context):
    inputs = event['data']
    result = []
    for input in inputs:
        serialized_input = ','.join(map(str, inputs[0]))
        response = runtime.invoke_endpoint(EndpointName= ENDPOINT_NAME,
                                          ContentType='text/csv',
                                          Body=serialized_input)
        result.append(response['Body'].read().decode().strip())
    
    return result

Input_json = {'data':[[0.324999988,0.324999988,0.324999988,0.324999988,20000.0],[0.324999988,0.324999988,0.324999988,0.324999988,20000.0],[0.324999988,0.324999988,0.324999988,0.324999988,20000.0]]
}

result = lambda_handler(Input_json, _)
result


['0.36412209272384644', '0.36412209272384644', '0.36412209272384644']

Calling API through API Gateway

In [55]:
import requests

API_ENDPOINT = "https://m1vfb49byk.execute-api.ap-southeast-1.amazonaws.com/xgbmodel"

json = {'data':[[0.324999988,0.324999988,0.324999988,0.324999988,20000.0],[0.324999988,0.324999988,0.324999988,0.324999988,20000.0],[0.324999988,0.324999988,0.324999988,0.324999988,20000.0]]
}

r = requests.post(url = API_ENDPOINT, json = json)

In [57]:
print(f"Status Code: {r.status_code}, Response: {r.json()}")

Status Code: 200, Response: ['0.36412209272384644', '0.36412209272384644', '0.36412209272384644']
